# JUMAN++のインストール

In [1]:
!wget http://lotus.kuee.kyoto-u.ac.jp/nl-resource/jumanpp/jumanpp-1.02.tar.xz
!tar xJvf jumanpp-1.02.tar.xz
%cd jumanpp-1.02
!./configure
!make
!sudo make install
%cd /content

--2025-01-08 12:49:20--  http://lotus.kuee.kyoto-u.ac.jp/nl-resource/jumanpp/jumanpp-1.02.tar.xz
Resolving lotus.kuee.kyoto-u.ac.jp (lotus.kuee.kyoto-u.ac.jp)... 130.54.208.131
Connecting to lotus.kuee.kyoto-u.ac.jp (lotus.kuee.kyoto-u.ac.jp)|130.54.208.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 631893448 (603M) [application/x-xz]
Saving to: ‘jumanpp-1.02.tar.xz’

jumanpp-1.02.tar.xz 100%[===================>] 602.62M  2.24MB/s    in 3m 51s  

2025-01-08 12:53:11 (2.61 MB/s) - ‘jumanpp-1.02.tar.xz’ saved [631893448/631893448]

jumanpp-1.02/
jumanpp-1.02/dict-build/
jumanpp-1.02/jumanpp-resource/
jumanpp-1.02/sample/
jumanpp-1.02/script/
jumanpp-1.02/Makefile.am
jumanpp-1.02/configure
jumanpp-1.02/configure.ac
jumanpp-1.02/aclocal.m4
jumanpp-1.02/Makefile.in
jumanpp-1.02/COPYING
jumanpp-1.02/INSTALL
jumanpp-1.02/compile
jumanpp-1.02/config.guess
jumanpp-1.02/config.sub
jumanpp-1.02/depcomp
jumanpp-1.02/install-sh
jumanpp-1.02/missing
jumanpp-1.02/README

In [2]:
!pip install -U pyknp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.4 MB/s eta 0:00:00


In [3]:
!echo 'こんにちは，今日は晴れです' | jumanpp

こんにちは こんにちは こんにちは 感動詞 12 * 0 * 0 * 0 "代表表記:こんにちは/こんにちは"
， ， ， 特殊 1 読点 2 * 0 * 0 NIL
今日 こんにち 今日 名詞 6 時相名詞 10 * 0 * 0 "代表表記:今日/こんにち カテゴリ:時間"
@ 今日 きょう 今日 名詞 6 時相名詞 10 * 0 * 0 "代表表記:今日/きょう カテゴリ:時間"
は は は 助詞 9 副助詞 2 * 0 * 0 NIL
晴れ はれ 晴れる 動詞 2 * 0 母音動詞 1 基本連用形 8 "代表表記:晴れる/はれる 自他動詞:他:晴らす/はらす 反義:動詞:曇る/くもる"
です です です 助動詞 5 * 0 無活用型 26 基本形 2 NIL
EOS


In [4]:
from pyknp import Juman
jumanpp = Juman()   # default is JUMAN++: Juman(jumanpp=True). if you use JUMAN, use Juman(jumanpp=False)
result = jumanpp.analysis("４５%です")
for mrph in result.mrph_list(): # 各形態素にアクセス
    print("見出し:%s, 読み:%s, 原形:%s, 品詞:%s, 品詞細分類:%s, 活用型:%s, 活用形:%s, 意味情報:%s, 代表表記:%s" \
            % (mrph.midasi, mrph.yomi, mrph.genkei, mrph.hinsi, mrph.bunrui, mrph.katuyou1, mrph.katuyou2, mrph.imis, mrph.repname))

見出し:４５, 読み:４５, 原形:４５, 品詞:名詞, 品詞細分類:数詞, 活用型:*, 活用形:*, 意味情報:カテゴリ:数量, 代表表記:
見出し:%, 読み:%, 原形:%, 品詞:未定義語, 品詞細分類:その他, 活用型:*, 活用形:*, 意味情報:品詞推定:名詞, 代表表記:
見出し:です, 読み:です, 原形:だ, 品詞:判定詞, 品詞細分類:*, 活用型:判定詞, 活用形:デス列基本形, 意味情報:NIL, 代表表記:


# クラスの定義

In [25]:
from pyknp import Juman
class RecommendRecipeMachine:
  def __init__(self):
    # 状態の種類
    # 'ASK_NAME'：ユーザの名前を質問（初期状態）
    # 'ASK_PEOPLE'：人数を質問
    # 'ASK_ENERGY'：今日のやる気を質問
    # 'ASK_INGREDIENT'：使いたい食材を質問
    # 'COMPLETION'：完了（終了状態）
    self.states = ['ASK_NAME', 'ASK_REQUEST','ASK_PEOPLE','ASK_ENERGY','ASK_INGREDIENT', 'COMPLETION']
    # 初期状態
    self.current_state = 'ASK_NAME'
    # 聞き取りした情報を保持するためのインスタンス変数
    self.name = None
    self.request = None
    self.people = None
    self.energy = None
    self.ingredient = None
    # 遷移状態の一覧
    # dict の value はインスタンスメソッドを呼び出している
    self.transitions = {
      # 'ASK_NAME' が呼び出されたときには　'handle_ask_name()' 呼び出される
      'ASK_NAME': self.handle_ask_name,
      'ASK_REQUEST': self.handle_ask_request,
      'ASK_PEOPLE': self.handle_ask_people,
      'ASK_ENERGY': self.handle_ask_energy,
      'ASK_INGREDIENT': self.handle_ask_ingredient,
    }

  # 入力された氏名を処理
  def handle_ask_name(self, user_input):
    # ユーザの入力をインスタンス変数　'name' に代入
    Jumanpp=Juman()
    result=Jumanpp.analysis(user_input)

    for mrph in result.mrph_list():
      if mrph.hinsi=="名詞" :
        self.name=mrph.genkei
        return f"{self.name}さん、今日の献立を考えるのをお手伝いします！どのようなものが食べたいですか？", 'ASK_REQUEST'
      else:
        self.name=user_input
        return "こんにちは！今日の献立を考えるのをお手伝いします！どのようなものが食べたいですか？", 'ASK_REQUEST'

  # 入力されたリクエストを保存（ここに既に人数、使いたい食材が含まれるときASKPEOPLEとASKINGREDIENTをスキップ）
  def handle_ask_request(self, user_input):
    self.request= user_input
    Jumanpp=Juman()
    result=Jumanpp.analysis(self.request)
    request_word=list()
    word_number=0
    count=0
    for mrph in result.mrph_list():
      request_word.append(mrph.genkei)
      count+=1
      if mrph.genkei=="使う" :
        word_number=count
        self.ingredient=request_word[word_number-3]
    if self.people==None:
      return "わかりました！何人でご飯を食べる予定ですか？", 'ASK_PEOPLE'
    else:
      return "わかりました！今日はどれくらいお料理頑張れそうですか？やる気レベルが何％かでお答えください！", 'ASK_ENERGY'

  # 入力された人数を処理
  def handle_ask_people(self, user_input):
    Jumanpp=Juman()
    result=Jumanpp.analysis(user_input)
    people=list()
    people_number=0
    count=0
    for mrph in result.mrph_list():
      people.append(mrph.genkei)
      count+=1
      if mrph.bunrui=="名詞性名詞助数辞" and mrph.imis.endswith("カテゴリ:人"):
        people_number=count

    if not people_number==0:
      ninzu=people[people_number-2]
      kanji=["","一","二","三","四","五","六","七","八","九","十"]
      if ninzu in kanji:
        ninzu=kanji.index(ninzu)
      else:
        ninzu=int(ninzu)
    else:
      ninzu=int(user_input)
    self.people = ninzu
    return f"{ninzu}人ですね！では、今日はどれくらいお料理頑張れそうですか？やる気レベルが何％か、数字でお答えください！", 'ASK_ENERGY'

  def handle_ask_energy(self,user_input):
    Jumanpp=Juman()
    result=Jumanpp.analysis(user_input)
    for mrph in result.mrph_list():
     if mrph.bunrui=="数詞":
        energy_level=int(mrph.genkei)

    if energy_level<20:
      self.energy = "低"
      return "今日もお疲れさまでした。最後に使いたい食材はありますか？", 'ASK_INGREDIENT'
    elif self.ingredient==None:
      if energy_level<40:
        self.energy = "中"
        return "わかりました！最後に、今日使いたい食材はありますか？", 'ASK_INGREDIENT'
      elif energy_level<80:
        self.energy = "高"
        return "わかりました！最後に、今日使いたい食材はありますか？", 'ASK_INGREDIENT'
      else:
        self.energy = "最高"
        return "わかりました！楽しくお料理しましょう！今日使いたい食材はありますか？", 'ASK_INGREDIENT'
    else:
      return "わかりました！あなたにお勧めの献立はこちらです！", 'COMPLETION'

  # 入力された食材を処理
  def handle_ask_ingredient(self,user_input):
    if user_input=="ない":
      self.ingredient = None
    else:
      self.ingredient = user_input
    return "ありがとうございます！あなたにお勧めの献立はこちらです！", 'COMPLETION'


  # システムの応答を取得　ここを変更してフレームベースにする
  def get_response(self, user_input):
    # 現在の状態 'current_state' に基づいてインスタンスメソッドが呼び出される
    # その際の引数としてユーザの入力 'user_input' が渡される
    # 戻り値はシステムの応用と次の状態
    response, next_state = self.transitions[self.current_state](user_input)
    # システムの状態を更新
    self.current_state = next_state
    return response

# 得られた情報からおすすめの献立を出力する

In [6]:
!pip install transformers

In [7]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import numpy as np

# 質問エンコーダとトークナイザーの読み込み
question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# コンテキストエンコーダとトークナイザーのロード
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-multiset-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-multiset-base')

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [9]:
import pandas as pd
df=pd.read_pickle("/content/recipe2.pickle")
df.head()

,title,url,times,calorie,sentence,ingredients
0,なすと鶏肉のドライカレー,https://www.kyounoryouri.jp/recipe/3738_,15.0,750.0,なすは角切りにして全体を炒めるとクタッとせずに食感も残り、油もよくからみます。ご飯はもちろん...,"なす, 鶏もも肉, ピーマン, にんにく, しょうが, カレー粉, オイスターソース, ご飯..."
1,牛丼,https://www.kyounoryouri.jp/recipe/4178_,10.0,670.0,牛肉、たまねぎのうまみとしょうゆの風みで、ご飯がすすみます。,"ご飯, 牛薄切り肉, たまねぎ, 温泉卵, 粉ざんしょう, 白ごま"
2,たけのこご飯,https://www.kyounoryouri.jp/recipe/8400_,45.0,310.0,アクぬきは、生のたけのこを切って大根おろし汁に浸すだけ。たけのこ本来の野性味が感じられます。,"新たけのこ, 米, 大根, 油揚げ, 木の芽, 塩"
3,もやしの中華丼,https://www.kyounoryouri.jp/recipe/2849_,15.0,400.0,マーボー豆腐のもやし版は、シャキシャキの歯ごたえとピリっとした辛さが、ご飯にぴったりです。め...,"もやし, 豚ひき肉, ねぎ, にんにく, 豆板醤, 粉ざんしょう, ご飯, サラダ油, かた..."
4,ツナの和風炊き込みご飯,https://www.kyounoryouri.jp/recipe/3488_,30.0,340.0,安くて手軽なツナ缶で、簡単、おいしい炊き込みご飯をつくります。細ねぎとしょうがたっぷりで召し...,"米, 水, ツナ, 細ねぎ, しょうが, 酒, しょうゆ"


In [21]:
df_low=df.query('times <= 15')
df_middle=df.query('times <= 30')
df_high=df.query('times > 30 and times <=60')
df_max=df.query('times > 60')

In [16]:
df_max.tail()

,title,url,times,calorie,sentence,ingredients
2206,関東炊き,https://www.kyounoryouri.jp/recipe/19390_,180.0,550.0,下ごしらえを丁寧にすれば、あとは煮るだけ。みんなで鍋の中をのぞきながら、次は何を食べようと話...,"大根, じゃがいも, 牛すじ, ちくわ, こんにゃく, 厚揚げ, ごぼう天, 卵, 練りがら..."
2231,本格おでん,https://www.kyounoryouri.jp/recipe/14030_,240.0,2640.0,寒くなる季節にうれしいおでん。手づくりのさつま揚げとがんもどきが絶品です。煮ては冷ます、を繰...,"大根, 米の研ぎ汁, しらたき, 卵, はんぺん, ちくわぶ, 溶きがらし, 七味とうがらし..."
2253,新たまねぎのスープ,https://www.kyounoryouri.jp/recipe/8485_,120.0,850.0,自然のチキンブイヨンと、新たまねぎのうまみの間合いを、大麦がつなぎます。,"新たまねぎ, エクストラバージンオリーブ油, 昆布, 梅干し, ローリエ, 塩, 大麦, イ..."
2341,ポリ袋みそ,https://www.kyounoryouri.jp/recipe/21211_,210.0,1760.0,ポリ袋でつくれば、かめやたるなど専用の容器も必要なく場所をとりません。大豆を戻さずにゆでる、...,"大豆, 米こうじ, 粗塩"
2388,五目豆,https://www.kyounoryouri.jp/recipe/4077_,90.0,1610.0,数種類の根菜を使っているので、食卓にこの一品が加わることで、栄養バランスもよくなります。,"大豆, れんこん, ごぼう, 干ししいたけ, にんじん, こんにゃく, 昆布, だしパック,..."


# 対話システム

In [26]:
# 対話システムのインスタンスを作成
dialog_system =RecommendRecipeMachine()

# 最初のシステムからの問いかけ
print("こんだてくん: こんにちは，こんだてくんです！")
print("こんだてくん: お名前を教えてください！")

# 状態が 'COMPLETION' となるまで対話を継続
while dialog_system.current_state != 'COMPLETION':
  user_input = input('あなた: ')
  response = dialog_system.get_response(user_input)
  print('こんだてくん: ', response)


# リクエスト
request = dialog_system.request
# コンテキスト
if dialog_system.energy=="低":
  df_ingredients= df_low[df_low['ingredients'].str.contains(dialog_system.ingredient, na=False)]
elif dialog_system.energy=="中":
  df_ingredients= df_middle[df_middle['ingredients'].str.contains(dialog_system.ingredient, na=False)]
elif dialog_system.energy=="高":
  df_ingredients= df_high[df_high['ingredients'].str.contains(dialog_system.ingredient, na=False)]
elif dialog_system.energy=="最高":
  df_ingredients= df_max[df_max['ingredients'].str.contains(dialog_system.ingredient, na=False)]
else:
  df_ingredients= df[df['ingredients'].str.contains(dialog_system.ingredient, na=False)]
contexts = df_ingredients.loc[:,"sentence"]

# 質問をトークン化
request_inputs = question_tokenizer(request, return_tensors="pt")
# 質問エンコーダーを使って質問の分散表現取得し，numpy に変換
question_embedding = question_encoder(**request_inputs).pooler_output.detach().numpy()

# コンテキストをエンコードした分散表現を格納するリスト
context_embedding_list = []
# コンテキストを一つずつエンコードする
for context in contexts:
  # コンテキストをトークン化
  context_inputs = context_tokenizer(context, return_tensors="pt")
  # コンテキストエンコーダーを使ってコンテキストの分散表現取得し，numpy に変換
  context_embedding = context_encoder(**context_inputs).pooler_output.detach().numpy()
  context_embedding_list.append(context_embedding)

# 内積で分散表現の類似度を計算
dot_products = []
for context_embedding in context_embedding_list:
  dot_product = np.dot(question_embedding, context_embedding.T)
  dot_products.append(dot_product.item())

# 類似度が高い順にソート
sorted_indices = np.argsort(dot_products)[::-1]
if sorted_indices.size > 0:  # Check if any recipes were found
    result = df_ingredients.iloc[sorted_indices[0]]["title"]
    url=df_ingredients.iloc[sorted_indices[0]]["url"]
    print("今日のあなたにはこれがおすすめ！：",result)
    print("URL:",url)
    print("その他のおすすめはこちら")
    for i in range(1, 6):
        if i < sorted_indices.size:
            result = df_ingredients.iloc[sorted_indices[i]]["title"]
            url=df_ingredients.iloc[sorted_indices[i]]["url"]
            print(i,":",result,url)
        else:
            break
else:
    result = "お探しの食材を使ったレシピは見つかりませんでした。"  # Handle case of no recipes found
    print(result)

こんだてくん: こんにちは，こんだてくんです！
こんだてくん: お名前を教えてください！
あなた: みわ
こんだてくん:  みわさん、今日の献立を考えるのをお手伝いします！どのようなものが食べたいですか？
あなた: 辛いもの
こんだてくん:  わかりました！何人でご飯を食べる予定ですか？
あなた: 6人
こんだてくん:  6人ですね！では、今日はどれくらいお料理頑張れそうですか？やる気レベルが何％か、数字でお答えください！
あなた: 15
こんだてくん:  今日もお疲れさまでした。最後に使いたい食材はありますか？
あなた: ねぎ
こんだてくん:  ありがとうございます！あなたにお勧めの献立はこちらです！
今日のあなたにはこれがおすすめ！： ねぎと豆腐のスープ
URL: https://www.kyounoryouri.jp/recipe/2138_
その他のおすすめはこちら
1 : みぞれすき焼き https://www.kyounoryouri.jp/recipe/14071_
2 : 鶏肉のから揚げと野菜のお吸い物 https://www.kyounoryouri.jp/recipe/16322_
3 : たけのこと春キャベツのスープ https://www.kyounoryouri.jp/recipe/2264_
4 : 鶏ひき肉の親子丼 https://www.kyounoryouri.jp/recipe/3327_
5 : ゴーヤーと桜えびのかき揚げ https://www.kyounoryouri.jp/recipe/17013_
